In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models # add models to the list
from torchvision.utils import make_grid
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")


import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision.utils import save_image

In [2]:
torch.manual_seed(100)

In [3]:
print(torch.__version__)

2.1.1+cpu


In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TRAIN_DIR = "data/train"
VAL_DIR = "data/val"
LEARNING_RATE = 2e-4
BATCH_SIZE = 32 #Use a number that can divide the total of photos- (here 961)
NUM_WORKERS = 2
IMAGE_SIZE = 256
CHANNELS_IMG = 3
L1_LAMBDA = 100
LAMBDA_GP = 10
NUM_EPOCHS = 30 #we suggest over 100
LOAD_MODEL = False
SAVE_MODEL = False
CHECKPOINT_DISC = "disc.pth.tar"
CHECKPOINT_GEN = "gen.pth.tar"

In [6]:
#Discriminator model for Pix2Pix

class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(CNNBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, stride, 1, bias=False, padding_mode="reflect"),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels * 2,features[0],kernel_size=4,stride=2,padding=1,padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(
                CNNBlock(in_channels, feature, stride=1 if feature == features[-1] else 2),
            )
            in_channels = feature

        layers.append(
            nn.Conv2d(
                in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"
            ),
        )

        self.model = nn.Sequential(*layers)

    def forward(self, x, y):
        x = torch.cat([x, y], dim=1)
        x = self.initial(x)
        return self.model(x)

In [7]:
#Generator model for Pix2Pix
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act="relu", use_dropout=False):
        super(Block, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False, padding_mode="reflect")
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act == "relu" else nn.LeakyReLU(0.2),
        )

        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)
        self.down = down

    def forward(self, x):
        x = self.conv(x)
        return self.dropout(x) if self.use_dropout else x


class Generator(nn.Module):
    def __init__(self, in_channels=3, features=64):
        super().__init__()
        self.initial_down = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )
        self.down1 = Block(features, features * 2, down=True, act="leaky", use_dropout=False)
        self.down2 = Block(features * 2, features * 4, down=True, act="leaky", use_dropout=False)
        self.down3 = Block(features * 4, features * 8, down=True, act="leaky", use_dropout=False)
        self.down4 = Block(features * 8, features * 8, down=True, act="leaky", use_dropout=False)
        self.down5 = Block(features * 8, features * 8, down=True, act="leaky", use_dropout=False)
        self.down6 = Block(features * 8, features * 8, down=True, act="leaky", use_dropout=False)
        self.bottleneck = nn.Sequential(
            nn.Conv2d(features * 8, features * 8, 4, 2, 1,padding_mode="reflect"), nn.ReLU()
        )

        self.up1 = Block(features * 8, features * 8, down=False, act="relu", use_dropout=True)
        self.up2 = Block(features * 8 * 2, features * 8, down=False, act="relu", use_dropout=True)
        self.up3 = Block(features * 8 * 2, features * 8, down=False, act="relu", use_dropout=True)
        self.up4 = Block(features * 8 * 2, features * 8, down=False, act="relu", use_dropout=False)
        self.up5 = Block(features * 8 * 2, features * 4, down=False, act="relu", use_dropout=False)
        self.up6 = Block(features * 4 * 2, features * 2, down=False, act="relu", use_dropout=False)
        self.up7 = Block(features * 2 * 2, features, down=False, act="relu", use_dropout=False)
        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(features * 2, in_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        d1 = self.initial_down(x)
        d2 = self.down1(d1)
        d3 = self.down2(d2)
        d4 = self.down3(d3)
        d5 = self.down4(d4)
        d6 = self.down5(d5)
        d7 = self.down6(d6)
        bottleneck = self.bottleneck(d7)
        up1 = self.up1(bottleneck)
        up2 = self.up2(torch.cat([up1, d7], 1))
        up3 = self.up3(torch.cat([up2, d6], 1))
        up4 = self.up4(torch.cat([up3, d5], 1))
        up5 = self.up5(torch.cat([up4, d4], 1))
        up6 = self.up6(torch.cat([up5, d3], 1))
        up7 = self.up7(torch.cat([up6, d2], 1))
        return self.final_up(torch.cat([up7, d1], 1))

In [8]:
#Train function for Pix2Pix

torch.backends.cudnn.benchmark = True


def train_fn(disc, gen, loader, opt_disc, opt_gen, l1_loss, bce, g_scaler, d_scaler,):
    loop = tqdm(loader, leave=True)

    for idx, (x, y) in enumerate(loop):
        x = x.to(DEVICE)
        y = y.to(DEVICE)

        # Train Discriminator
        with torch.cuda.amp.autocast():
            y_fake = gen(x)
            D_real = disc(x, y)
            D_fake = disc(x, y_fake.detach())
            D_real_loss = bce(D_real, torch.ones_like(D_real))
            D_fake_loss = bce(D_fake, torch.zeros_like(D_fake))
            D_loss = (D_real_loss + D_fake_loss) / 2

        disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train generator
        with torch.cuda.amp.autocast():
            D_fake = disc(x, y_fake)
            G_fake_loss = bce(D_fake, torch.ones_like(D_fake))
            L1 = l1_loss(y_fake, y) * L1_LAMBDA
            G_loss = G_fake_loss + L1

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        if idx % 10 == 0:
            loop.set_postfix(
                D_real=torch.sigmoid(D_real).mean().item(),
                D_fake=torch.sigmoid(D_fake).mean().item(),
            )

In [9]:
# Utils for Pix2Pix

def save_some_examples(gen, val_loader, epoch, folder):
  ##ADDITION
    y_pred = []
  ##

    x, y = next(iter(val_loader))
    x, y = x.to(DEVICE), y.to(DEVICE)
    gen.eval()
    with torch.no_grad():
        y_fake = gen(x)
        y_fake = y_fake * 0.5 + 0.5  # remove normalization#

        ##ADDITION
        y_prediction = y_fake.cpu().numpy() #convert pytorch tensor to numpy
        y_pred.append(y_prediction)
        ##

        save_image(y_fake, folder + f"/y_gen_{epoch}.png")
        save_image(x * 0.5 + 0.5, folder + f"/input_{epoch}.png")
        if epoch == 1:
            save_image(y * 0.5 + 0.5, folder + f"/label_{epoch}.png")
    gen.train()

    ##ADDITION
    ypred = np.array(y_pred)
    #print(ypred)
    np.save("ypred.npy", ypred)
    ##


def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=config.DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

**SUPERSAMPLING**

Create the training set

In [10]:
#import data
df = pd.read_csv('x train db3.csv')
df.head()

,id,fuel,per time cycling,super and rest
0,1,0.05689,0.15860,0
1,2,0.05689,0.15860,0
2,3,0.05689,0.14560,0
3,4,0.05689,0.14222,0
4,5,0.05689,0.14222,0


In [11]:
df.columns

Index(['id', 'fuel', 'per time cycling', 'super and rest'], dtype='object')

In [12]:
#Transform to numpy
data=np.array(df[['id', 'fuel', 'per time cycling', 'super and rest']])
data.shape

(250000, 4)

In [13]:
data=np.transpose(data)

data= data.reshape(4,500,500)
data.shape


(4, 500, 500)

In [14]:
#Divide into set

w=500 #width
h=500 #height
s=256 #size
step=32 #step


idlist=[]

for i in range(s//2-1,w-s//2,step):
    for j in range(s//2-1,h-s//2,step):
        idlist.append(data[0,i,j])


dataclip=np.empty((0,3,256,256))

for id in idlist:
    #print(id)

    row=int(id//h)
    #print(row)

    col=int(id-h*row -1)
    #print(col)

    #print(s)
    minRow = row-s//2+1
    maxRow = row+s//2+1
    minCol = col-s//2+1
    maxCol = col+s//2+1

    #remove id column because we do not need it further
    sample= data[1:,minRow:maxRow,minCol:maxCol]

    #print(sample.shape)
    dataclip=np.append(dataclip,[sample],axis=0)

In [15]:
dataclip.shape

(64, 3, 256, 256)

Repeat the process for labels

In [16]:
#import data
df = pd.read_csv('y label db3.csv')
df.head()

,id,per green
0,1,6.85954
1,2,6.85954
2,3,6.85954
3,4,6.85954
4,5,6.85954


In [17]:
df.columns

Index(['id', 'per green '], dtype='object')

In [18]:
#Transform to numpy
data=np.array(df)
data.shape

(250000, 2)

In [19]:
data=np.transpose(data)

data= data.reshape(2,500,500)
data.shape

(2, 500, 500)

In [20]:
#Divide into set

w=500 #width
h=500 #height
s=256 #size
step=32 #step


idlisty=[]

for i in range(s//2-1,w-s//2,step):
    for j in range(s//2-1,h-s//2,step):
        idlisty.append(data[0,i,j])

print(idlisty)



[63628.0, 63660.0, 63692.0, 63724.0, 63756.0, 63788.0, 63820.0, 63852.0, 79628.0, 79660.0, 79692.0, 79724.0, 79756.0, 79788.0, 79820.0, 79852.0, 95628.0, 95660.0, 95692.0, 95724.0, 95756.0, 95788.0, 95820.0, 95852.0, 111628.0, 111660.0, 111692.0, 111724.0, 111756.0, 111788.0, 111820.0, 111852.0, 127628.0, 127660.0, 127692.0, 127724.0, 127756.0, 127788.0, 127820.0, 127852.0, 143628.0, 143660.0, 143692.0, 143724.0, 143756.0, 143788.0, 143820.0, 143852.0, 159628.0, 159660.0, 159692.0, 159724.0, 159756.0, 159788.0, 159820.0, 159852.0, 175628.0, 175660.0, 175692.0, 175724.0, 175756.0, 175788.0, 175820.0, 175852.0]


In [21]:
ylabel=np.empty((0,1,256,256))

for id in idlisty:
    #print(id)

    row=int(id//h)
    #print(row)

    col=int(id-h*row -1)
    #print(col)

    #print(s)
    minRow = row-s//2+1
    maxRow = row+s//2+1
    minCol = col-s//2+1
    maxCol = col+s//2+1

    #remove id column because we do not need it further
    sample= data[1:,minRow:maxRow,minCol:maxCol]

    #print(sample.shape)
    ylabel=np.append(ylabel,[sample],axis=0)

In [22]:
ylabel.shape

(64, 1, 256, 256)

In [23]:
#recreating 3 channels in total
rgb = np.hstack((ylabel,ylabel,ylabel))
rgb.shape

(64, 3, 256, 256)

Create the test set / validation set

In [24]:
#import data
dft = pd.read_csv('test xn3.csv')
dft.head()

,id,per green,per time cycling,super and rest,fuel
0,1,10.39321,0.04746,0,0.0318
1,2,10.39321,0.04746,0,0.0318
2,3,10.39321,0.04746,0,0.0318
3,4,10.39321,0.04746,0,0.0318
4,5,10.39321,0.04746,0,0.0318


In [25]:
dft.columns

Index(['id', 'per green', 'per time cycling', 'super and rest', 'fuel'], dtype='object')

In [26]:
#Transform to numpy
datatest=np.array(dft[['id','per time cycling', 'super and rest', 'fuel']])
datatest.shape

datatest=np.transpose(datatest)

datatest= datatest.reshape(4,500,500)
datatest.shape

(4, 500, 500)

In [27]:
#Divide into set

w=500 #width
h=500 #height
s=256 #size
step=32 #step


idlisttest=[]

for i in range(s//2-1,w-s//2,step):
    for j in range(s//2-1,h-s//2,step):
        idlisttest.append(datatest[0,i,j])

print(idlisttest)



[63628.0, 63660.0, 63692.0, 63724.0, 63756.0, 63788.0, 63820.0, 63852.0, 79628.0, 79660.0, 79692.0, 79724.0, 79756.0, 79788.0, 79820.0, 79852.0, 95628.0, 95660.0, 95692.0, 95724.0, 95756.0, 95788.0, 95820.0, 95852.0, 111628.0, 111660.0, 111692.0, 111724.0, 111756.0, 111788.0, 111820.0, 111852.0, 127628.0, 127660.0, 127692.0, 127724.0, 127756.0, 127788.0, 127820.0, 127852.0, 143628.0, 143660.0, 143692.0, 143724.0, 143756.0, 143788.0, 143820.0, 143852.0, 159628.0, 159660.0, 159692.0, 159724.0, 159756.0, 159788.0, 159820.0, 159852.0, 175628.0, 175660.0, 175692.0, 175724.0, 175756.0, 175788.0, 175820.0, 175852.0]


In [28]:
datacliptest=np.empty((0,3,256,256))
##UPDATE
idvaltest=np.empty((0,256,256))

for id in idlisttest:
    #print(id)

    row=int(id//h)
    #print(row)

    col=int(id-h*row -1)
    #print(col)

    #print(s)
    minRow = row-s//2+1
    maxRow = row+s//2+1
    minCol = col-s//2+1
    maxCol = col+s//2+1

    #remove id column because we do not need it further
    sample= datatest[1:,minRow:maxRow,minCol:maxCol]
    idval=datatest[0,minRow:maxRow,minCol:maxCol]

    #print(sample.shape)
    datacliptest=np.append(datacliptest,[sample],axis=0)
    idvaltest=np.append(idvaltest,[idval],axis=0)

In [29]:
datacliptest.shape

(64, 3, 256, 256)

Create the test label

In [30]:
#Transform to numpy
datatesty=np.array(dft[['id', 'per green']])
datatesty.shape

datatesty=np.transpose(datatesty)

datatesty= datatesty.reshape(2,500,500)
datatesty.shape

(2, 500, 500)

In [31]:
#Divide into set

w=500 #width
h=500 #height
s=256 #size
step=32 #step


idlisttesty=[]

for i in range(s//2-1,w-s//2,step):
    for j in range(s//2-1,h-s//2,step):
        idlisttesty.append(datatesty[0,i,j])

print(idlisttesty)

[63628.0, 63660.0, 63692.0, 63724.0, 63756.0, 63788.0, 63820.0, 63852.0, 79628.0, 79660.0, 79692.0, 79724.0, 79756.0, 79788.0, 79820.0, 79852.0, 95628.0, 95660.0, 95692.0, 95724.0, 95756.0, 95788.0, 95820.0, 95852.0, 111628.0, 111660.0, 111692.0, 111724.0, 111756.0, 111788.0, 111820.0, 111852.0, 127628.0, 127660.0, 127692.0, 127724.0, 127756.0, 127788.0, 127820.0, 127852.0, 143628.0, 143660.0, 143692.0, 143724.0, 143756.0, 143788.0, 143820.0, 143852.0, 159628.0, 159660.0, 159692.0, 159724.0, 159756.0, 159788.0, 159820.0, 159852.0, 175628.0, 175660.0, 175692.0, 175724.0, 175756.0, 175788.0, 175820.0, 175852.0]


In [32]:
ytest=np.empty((0,1,256,256))

for id in idlisttesty:
    #print(id)

    row=int(id//h)
    #print(row)

    col=int(id-h*row -1)
    #print(col)

    #print(s)
    minRow = row-s//2+1
    maxRow = row+s//2+1
    minCol = col-s//2+1
    maxCol = col+s//2+1

    #remove id column because we do not need it further
    sample= datatesty[1:,minRow:maxRow,minCol:maxCol]

    #print(sample.shape)
    ytest=np.append(ytest,[sample],axis=0)

In [33]:
ytest.shape

(64, 1, 256, 256)

In [34]:
#recreating 3 channels in total
rgbtest = np.hstack((ytest,ytest,ytest))
rgbtest.shape

(64, 3, 256, 256)

**TRAIN THE MODEL**

In [35]:
#Configurations

disc = Discriminator(in_channels=3).to(DEVICE)
gen = Generator(in_channels=3, features=64).to(DEVICE)
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999),)
opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
BCE = nn.BCEWithLogitsLoss()
L1_LOSS = nn.L1Loss()

In [36]:
#Train the model
from torch.utils.data import TensorDataset, DataLoader

##ADDITION
tensor_x = torch.Tensor(dataclip) # transform to torch tensor
tensor_y = torch.Tensor(rgb)

train_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
##

train_loader = DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=NUM_WORKERS,)

g_scaler = torch.cuda.amp.GradScaler()
d_scaler = torch.cuda.amp.GradScaler()


for epoch in range(NUM_EPOCHS):
    train_fn(disc, gen, train_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler,)

100%|███████████████████████████████████████████████████████| 2/2 [00:48<00:00, 24.42s/it, D_fake=0.0691, D_real=0.937]


In [37]:
#Test the model

tensor_x = torch.Tensor(datacliptest) # transform to torch tensor
tensor_y = torch.Tensor(rgbtest)

val_dataset = TensorDataset(tensor_x,tensor_y) # create your datset

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

save_some_examples(gen, val_loader, 4, folder="C:\\Users\\15809\\Desktop\\pix final") #Remember to choose the epoch and creat a folder on the desktop

**SAVE THE RESULTS**

In [38]:
#Import the saved data
y= np.load("ypred.npy")
y2=y.reshape(BATCH_SIZE, 3, -1)

idvaltest2=idvaltest.reshape(2,BATCH_SIZE,1,256,256) # total number/batch size=31
###UPDATE
idvaltest3=idvaltest2[1].reshape(32,-1)

In [39]:
for i in range(len(idvaltest3)):
    dfy=pd.DataFrame(idvaltest3[i,:],columns=['id'])
    dfy['ch1']= y2[i,0,:]
    dfy['ch2']= y2[i,1,:]
    dfy['ch3']= y2[i,2,:]
    dfy.to_csv(f'./_pred_{i}.csv',index=False)